# Principales funciones LCEL integradas para ejecutables

En LangChain, las funciones `.bind()` y `.assign()` son utilizadas para configurar y personalizar la ejecución de componentes:

- **.bind()**: permite asociar funciones o métodos a un objeto, permitiendo la integración de parámetros y configuraciones específicas para su ejecución.

- **.assign()**: se usa para asignar valores a los parámetros de entrada de un runnable, permitiendo la creación de flujos de trabajo más flexibles y personalizados.

Ambas funciones facilitan la construcción de cadenas de ejecución más complejas y adaptadas a las necesidades del usuario.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

In [6]:
prompt = ChatPromptTemplate.from_template("Dime algo curioso sobre {soccer_player}")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"soccer_player": "De Bruyne"})

'Kevin De Bruyne, el talentoso mediocampista belga del Manchester City, es conocido no solo por su habilidad en el campo, sino también por su impresionante visión de juego y precisión en los pases. Un dato curioso sobre él es que, a pesar de su éxito en el fútbol, tuvo un inicio complicado en su carrera. Fue rechazado por el Chelsea en 2012, y en lugar de rendirse, se trasladó a Alemania para jugar en el Werder Bremen. Su actuación allí llamó la atención del VfL Wolfsburg, donde realmente comenzó a destacar y a desarrollar su potencial, lo que finalmente lo llevó de regreso a la Premier League, donde se ha convertido en uno de los mejores mediocampistas del mundo. Este camino muestra su determinación y capacidad para superar obstáculos en su carrera.'

## Uso de .bind() para agregar argumentos a un Runnable en una cadena LCEL
* Por ejemplo, podemos agregar un argumento para detener la respuesta del modelo cuando llega a la palabra "De Bruyne":

In [7]:
chain = prompt | model.bind(stop=["City"]) | output_parser

chain.invoke({"soccer_player": "De Bruyne"})

'Kevin De Bruyne, el talentoso mediocampista belga, es conocido no solo por su habilidad en el campo, sino también por su impresionante visión de juego. Un dato curioso sobre él es que, antes de convertirse en una estrella en el Manchester '

## Uso de .bind() para llamar a una función OpenAI en una cadena LCEL

In [8]:
functions = [
    {
      "name": "soccerfacts",
      "description": "Curious facts about a soccer player",
      "parameters": {
        "type": "object",
        "properties": {
          "question": {
            "type": "string",
            "description": "The question for the curious facts about a soccer player"
          },
          "answer": {
            "type": "string",
            "description": "The answer to the question"
          }
        },
        "required": ["question", "answer"]
      }
    }
  ]

In [9]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "soccerfacts"}, functions= functions)
    | JsonOutputFunctionsParser()
)

In [10]:
chain.invoke(input={"soccer_player": "Mbappe"})

{'question': '¿Qué edad tenía Mbappé cuando debutó en la Ligue 1?',
 'answer': 'Mbappé debutó en la Ligue 1 a los 16 años.'}

**Nota:** La API de OpenAI ha dejado de lado algunas funciones en favor de herramientas. Consulta [aquí](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/openai_functions_agent/) para obtener más información.

## Uso de .bind() para adjuntar herramientas OpenAI
**Nota:** En la API de chat de OpenAI, las funciones ahora se consideran opciones heredadas que se han dejado de lado en favor de las herramientas. Si está creando agentes con modelos LLM de OpenAI, debería usar herramientas de OpenAI en lugar de funciones de OpenAI.

Si bien generalmente debe utilizar el método .bind_tools() para los modelos que invocan herramientas, también puede vincular argumentos específicos del proveedor directamente si desea un control de nivel inferior:

In [12]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [13]:
model = ChatOpenAI(model="gpt-3.5-turbo").bind(tools=tools)
model.invoke("¿Cómo es el clima en San Francisco, Nueva York y Los Ángeles?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gyqYtONTE9k5NlKUPpun9G3y', 'function': {'arguments': '{"location": "San Francisco"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_wtlShy8mpf0yZMzoruYmyvbO', 'function': {'arguments': '{"location": "New York"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_WSl8Mihw9KN3I74r5kVn6UIH', 'function': {'arguments': '{"location": "Los Angeles"}', 'name': 'get_current_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 93, 'total_tokens': 156, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d454128e-5f60-4685-a07a-3f5097f3cbbb-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 

## La función assign() permite agregar claves a una cadena
* Ejemplo: crearemos una clave llamada "operation_b" asignada a una función personalizada con un RunnableLambda.
* Comenzaremos con una cadena muy básica con solo RunnablePassthrough:

In [14]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

chain = RunnableParallel({"original_input": RunnablePassthrough()})

In [15]:
chain.invoke("clima")

{'original_input': 'clima'}

* Como puede ver, en este momento esta cadena solo asigna la entrada del usuario a la variable "original_input".
* Ahora agreguemos la nueva clave "uppercase" con la función de asignación.
* En la nueva clave "uppercase", usaremos un RunnableLambda con la función personalizada denominada `make_uppercase`

In [16]:
def make_uppercase(arg):
    return arg["original_input"].upper()

In [17]:
chain = RunnableParallel({"original_input": RunnablePassthrough()}).assign(uppercase=RunnableLambda(make_uppercase))

In [18]:
chain.invoke("clima")

{'original_input': 'clima', 'uppercase': 'CLIMA'}

* Como puede ver, la salida de la cadena ahora tiene 2 claves: original_input y uppercase.
* En la clave uppercase, podemos ver que la función `make_uppercase` se ha aplicado a la entrada del usuario.